In [ ]:
# 导入必要的包
import requests  # 发起请求
from bs4 import BeautifulSoup  # 解析网页数据
import re     # 使用正则表达式清洗文本
from tqdm import tqdm
import threading

In [ ]:
# 选取爬取目标
target = 'https://www.jx.la/book/4292/2713513.html'
# 发起请求
def get_context(chapters):
    contexts = ''
    # for url, subject in tqdm(chapters):
    for url, subject in tqdm(chapters):
        req = requests.get(url=url)
        req.encoding = 'utf-8'
        html = req.text                     # 获取请求后返回的文本
        bs = BeautifulSoup(html, 'lxml')    # 将文本传给bs工具
        # 根据html元素类型和id从文本中抽取需要的内容
        context = bs.find('div', id='content')
        # print(texts)  # 得到<div id='content'>中的所有内容
        context = context.text.replace('<br/>', '')
        context = re.sub('\s+', '', context)
        contexts += '\n' + subject + '\n' + context
        
    return contexts

In [ ]:
def get_content(content_url):
    req = requests.get(url=content_url)
    req.encoding = 'utf-8'
    html = req.text
    bs = BeautifulSoup(html, 'lxml')
    information = bs.find('div', id='info')
    title = information.text.split()[0]      # get title of novel
    chapters = bs.find('div', id='list')
    chapters = bs.findAll('dd')
    chapter_urls = [(target[:17]+chapte.a['href'], chapte.a.text)
        for chapte in chapters[12:]]
    
    return chapter_urls

target = 'https://www.jx.la/book/4292/'
chapters = get_content(target)

In [ ]:
'''
————————————————
版权声明：本文为CSDN博主「行者小朱」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/u012050154/article/details/80032072
'''
class MyThread(threading.Thread):
    def __init__(self, func, args=()):
        super(MyThread, self).__init__()
        self.func = func
        self.args = args
 
    def run(self):
        self.result = self.func(self.args)
 
    def get_result(self):
        threading.Thread.join(self) # 等待线程执行完毕
        try:
            return self.result
        except Exception:
            return None

In [ ]:
%%time
# 选取爬取目标
target = 'https://www.jx.la/book/4292/'
# 获取章节目录
chapters = get_content(target)

results = []

# cpu线程数
thread_nums = 12
# 把每个线程任务数量
chapters_length = len(chapters)
stepLength = chapters_length // thread_nums
tasks = []
for thread_num in range(thread_nums):
    start = thread_num * stepLength
    end = (thread_num + 1) * stepLength
    if end > chapters_length:
        end = chapters_length
    task = MyThread(get_context, (chapters[start: end]))
    task.start()
    tasks.append(task)

for task in tasks:
    results.append((thread_num, task.get_result()))
    
sorted(results, key=lambda x:x[0])

with open('修真四万年.txt', 'w') as f:
    for num, contexts in results:
        print(num)
        f.write(contexts)